In [14]:
import pandas as pd
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pymongo import MongoClient

df1 = pd.read_csv(r'C:\DS 220 Final Project\nasdaq.csv.csv')
df2 = pd.read_csv(r'C:\DS 220 Final Project\Nyse.csv.csv')

df = pd.concat([df1, df2], axis=0)

df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,AACG,ATA Creativity Global American Depositary Shares,$0.90,0.0001,0.011%,28800117.0,China,2008.0,3199,Real Estate,Other Consumer Services
1,AACI,Armada Acquisition Corp. I Common Stock,$11.8899,0.2599,2.235%,0.0,United States,2021.0,112,Finance,Blank Checks
2,AACIW,Armada Acquisition Corp. I Warrant,$0.14,0.0040,2.941%,0.0,United States,2021.0,121930,Finance,Blank Checks
3,AADI,Aadi Bioscience Inc. Common Stock,$1.9301,-0.0099,-0.51%,47392071.0,United States,NaN,2445,Health Care,Biotechnology: Pharmaceutical Preparations
4,AAGR,African Agriculture Holdings Inc. Common Stock,$0.3898,0.0100,2.633%,22556490.0,United States,2021.0,29983,Consumer Staples,Farming/Seeds/Milling


In [23]:
subset_df = df[(df['Market Cap'] > 1e12) | (df['Market Cap'] < 10000)]
ticks = subset_df['Symbol']
sorted_ticks = ticks.sort_values()
sorted_ticks.head()

1      AACI
2     AACIW
2      AACT
10     AAPL
28     ACAB
Name: Symbol, dtype: object

In [24]:
stock = yf.Ticker("MSFT")
stock.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [16]:
client = MongoClient("mongodb://localhost:27017/")
db = client["stocks"]
collection = db["stock_info"]

def insert_stock_info(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        collection.insert_one(info)
        print(f"Successfully inserted info for {ticker}")
    except Exception as e:
        print(f"Error inserting info for {ticker}: {e}")

for ticker in sorted_ticks:
    insert_stock_info(ticker)

num_documents = collection.count_documents({})
print(f"Number of documents in the collection: {num_documents}")

Successfully inserted info for AACI
Successfully inserted info for AACIW
Successfully inserted info for AACT
Successfully inserted info for AAPL
Successfully inserted info for ACAB
Successfully inserted info for ACABW
Successfully inserted info for ACACU
Successfully inserted info for ACACW
Successfully inserted info for ACP
Successfully inserted info for ADX
Successfully inserted info for AEFC
Successfully inserted info for AFAR
Successfully inserted info for AFARW
Successfully inserted info for AFB
Successfully inserted info for AFJK
Successfully inserted info for AFT
Successfully inserted info for AGD
Successfully inserted info for AHR
Successfully inserted info for AIB
Successfully inserted info for AIF
Successfully inserted info for AIMAU
Successfully inserted info for AITR
Successfully inserted info for AITRU
Successfully inserted info for ALCC
Successfully inserted info for ALCY
Successfully inserted info for ALSA
Successfully inserted info for ALSAR
Successfully inserted info f

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FEI%20%20%20%20%20%20%20%20%20%20%20?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FEI+++++++++++&crumb=aptgdMsxVde


Successfully inserted info for FEI           
Successfully inserted info for FEXD
Successfully inserted info for FFA
Successfully inserted info for FFC
Successfully inserted info for FGB
Successfully inserted info for FIACW
Successfully inserted info for FIF
Successfully inserted info for FLC
Successfully inserted info for FMY
Successfully inserted info for FNVT
Successfully inserted info for FORL
Successfully inserted info for FORLW
Successfully inserted info for FPF
Successfully inserted info for FPL
Successfully inserted info for FSD
Successfully inserted info for GAMC
Successfully inserted info for GAMCW
Successfully inserted info for GAQ
Successfully inserted info for GATE
Successfully inserted info for GBBK
Successfully inserted info for GBBKR
Successfully inserted info for GDL
Successfully inserted info for GDO
Successfully inserted info for GDST
Successfully inserted info for GECCI
Successfully inserted info for GECCM
Successfully inserted info for GECCO
Successfully inserted i

In [21]:
california_stocks = collection.find({"state": "CA"})
print("Stocks located in California:")
for stock in california_stocks:
    print(stock["symbol"])

Stocks located in California:
AAPL
AHR
ATNFW
BKHAU
BNIX
BNIXR
BNIXW
CCIF
CGON
CNGL
COOL
COOLW
DFP
DMA
EMCG
FFC
FLC
FORL
GOOG
GOOGL
IROH
IROHR
IROHU
IROHW
JQC
KIO
LCW
MCAC
MCACR
META
MNTN
NMHI
NMHIW
NVDA
PAI
PAXS
PCK
PCM
PCN
PCQ
PEGR
PEGRU
PFD
PFL
PFN
PFO
PGP
PHK
PLMI
PLMIW
PLMJ
PLMJU
PMF
PMGMW
PML
PMX
PNF
PNI
PPYA
PTY
PYN
PZC
RBRK
RCS
ROCL
ROCLW
SPKL
SPKLU
SPKLW
TEI
TRON
TSI
VMCA
WEA
WIA


In [22]:
cursor = collection.find({"symbol": {"$exists": True}, "previousClose": {"$exists": True}}).sort("previousClose")
print("Stocks sorted by previousClose:")
for stock in cursor:
    if "symbol" in stock and "previousClose" in stock:
        print(f"{stock['symbol']}: {stock['previousClose']}")
    else:
        print("Missing key 'symbol' or 'previousClose' for this stock:", stock)

Stocks sorted by previousClose:
SLNAW: 0.0071
ISPOW: 0.0081
ATNFW: 0.0085
UKOMW: 0.01
PUCKW: 0.0101
WINVW: 0.0101
AFARW: 0.013
EDBLW: 0.0185
BNIXW: 0.02
HSPOW: 0.02
PMGMW: 0.02
ATMCW: 0.0227
IMAQW: 0.028
GLLIW: 0.0288
RFACW: 0.03
JTAIW: 0.0301
NMHIW: 0.0303
WAVSW: 0.0311
BUJAW: 0.032
HHGCW: 0.0349
IROHW: 0.035
NOVVW: 0.035
DECAW: 0.0355
AOGOW: 0.0379
INAQW: 0.04
FIACW: 0.0401
PLTNW: 0.0444
DISTW: 0.0498
FORLW: 0.05
RWODW: 0.05
XFINW: 0.05
PLAOW: 0.0502
MLECW: 0.051
HAIAW: 0.053
GMFIW: 0.0599
IVCAW: 0.0649
ACACW: 0.07
GBBKR: 0.071
ISRLW: 0.071
NBSTW: 0.0769
BWAQW: 0.0777
ROCLW: 0.0801
TGAAW: 0.0817
ACABW: 0.088
SVIIW: 0.0903
KACLR: 0.0906
GLSTR: 0.0945
PTWOW: 0.0966
BHACW: 0.1
IBACR: 0.1
KRNLW: 0.1
NVACW: 0.1
ALSAR: 0.1075
RFACR: 0.108
ATMVR: 0.11
CFFSW: 0.11
BCSAW: 0.1126
NETDW: 0.1129
RENEW: 0.115
MRNOW: 0.117
OCAXW: 0.1175
GPACW: 0.12
MCACR: 0.12
MCAAW: 0.1246
IPXXW: 0.1251
BOWNR: 0.126
ZPTAW: 0.1298
BNIXR: 0.13
SVIIR: 0.13
BAYAR: 0.1339
CAPTW: 0.1389
IGTAW: 0.14
WTMAR: 0.14
AACIW: 0

In [19]:
pipeline = [
    {"$match": {"previousClose": {"$gt": 100}}},
    {"$group": {"_id": None, "average_previousClose": {"$avg": "$previousClose"}}}
]
result = list(collection.aggregate(pipeline))
if result:
    average_previousClose = result[0]["average_previousClose"]
    print(f"Average previousClose for stocks with previousClose > $100: {average_previousClose}")
else:
    print("No stocks found with previousClose > $100")

Average previousClose for stocks with previousClose > $100: 334.61
